In [ ]:
# sournce: https://www.kaggle.com/code/sahidvelji/meet-the-grandmasters 

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import json

import seaborn as sns
import matplotlib.pyplot as plt

from geopy.geocoders import Photon
from geopy.extra.rate_limiter import RateLimiter

from geopandas.tools import geocode
import folium
from folium import Marker
from folium.plugins import MarkerCluster

from IPython.display import display, HTML
        
KAGGLE_BASE_URL = 'https://kaggle.com/'
TWITTER_BASE_URL = 'https://twitter.com/'
GITHUB_BASE_URL = 'https://github.com/'
#TOP_N = 10

# modified
import os
def beep():
    os.system('afplay /System/Library/Sounds/Ping.aiff')

import pickle


In [ ]:
# # Datasets
datasets = pd.read_csv('./data/Datasets.csv', 
                       parse_dates=['CreationDate', 'LastActivityDate'], 
                       infer_datetime_format=True
                      )
dataset_versions = pd.read_csv('./data/DatasetVersions.csv', 
                               parse_dates=['CreationDate'], 
                               infer_datetime_format=True
                              )
# dataset_votes = pd.read_csv('/kaggle/input/meta-kaggle/DatasetVotes.csv', 
#                             parse_dates=['VoteDate'], infer_datetime_format=True
#                            )
# dataset_tags = pd.read_csv('/kaggle/input/meta-kaggle/DatasetTags.csv')
# data_sources = pd.read_csv('/kaggle/input/meta-kaggle/Datasources.csv', 
#                            parse_dates=['CreationDate'], infer_datetime_format=True
#                           )

# # Kernels
kernels = pd.read_csv('./data/Kernels.csv', 
                      parse_dates=['CreationDate', 'EvaluationDate',
                                   'MadePublicDate', 'MedalAwardDate'],
                      infer_datetime_format=True
                     )
kernel_versions = pd.read_csv('./data/KernelVersions.csv', 
                              parse_dates=['CreationDate', 'EvaluationDate'],
                              infer_datetime_format=True
                             )
# kernel_votes = pd.read_csv('/kaggle/input/meta-kaggle/KernelVotes.csv', 
#                            parse_dates=['VoteDate'], infer_datetime_format=True
#                           )
# kernel_version_comp = pd.read_csv('/kaggle/input/meta-kaggle/KernelVersionCompetitionSources.csv')
# kernel_version_out = pd.read_csv('/kaggle/input/meta-kaggle/KernelVersionOutputFiles.csv')
# kernel_version_datasets = pd.read_csv('/kaggle/input/meta-kaggle/KernelVersionDatasetSources.csv')
# kernel_langs = pd.read_csv('/kaggle/input/meta-kaggle/KernelLanguages.csv')
# kernel_version_kernel_sources = pd.read_csv('/kaggle/input/meta-kaggle/KernelVersionKernelSources.csv')
# kernel_tags = pd.read_csv('/kaggle/input/meta-kaggle/KernelTags.csv')

# # Forums
messages = pd.read_csv('./data/ForumMessages.csv', 
                       parse_dates=['PostDate', 'MedalAwardDate'], 
                       infer_datetime_format=True
                      )
topics = pd.read_csv('./data/ForumTopics.csv', 
                     parse_dates=['CreationDate', 'LastCommentDate'], 
                     infer_datetime_format=True
                    )
# forums = pd.read_csv('/kaggle/input/meta-kaggle/Forums.csv')
# forum_votes = pd.read_csv('/kaggle/input/meta-kaggle/ForumMessageVotes.csv', 
#                           parse_dates=['VoteDate'], infer_datetime_format=True
#                          )

# # Competitions
# teams = pd.read_csv('/kaggle/input/meta-kaggle/Teams.csv', 
#                     parse_dates=['ScoreFirstSubmittedDate', 'LastSubmissionDate',
#                                  'MedalAwardDate'], 
#                     infer_datetime_format=True
#                    )
# comps = pd.read_csv('/kaggle/input/meta-kaggle/Competitions.csv', 
#                     parse_dates=['EnabledDate', 'DeadlineDate', 
#                                  'ProhibitNewEntrantsDeadlineDate', 'TeamMergerDeadlineDate', 
#                                  'TeamModelDeadlineDate', 'ModelSubmissionDeadlineDate'], 
#                     infer_datetime_format=True
#                    )
# comp_tags = pd.read_csv('/kaggle/input/meta-kaggle/CompetitionTags.csv')
# team_membership = pd.read_csv('/kaggle/input/meta-kaggle/TeamMemberships.csv', 
#                               parse_dates=['RequestDate'], 
#                               infer_datetime_format=True
#                              )
# submissions = pd.read_csv('/kaggle/input/meta-kaggle/Submissions.csv', 
#                           parse_dates=['SubmissionDate', 'ScoreDate'], 
#                           infer_datetime_format=True
#                          )

# # Users
users = pd.read_csv('./data/Users.csv',
                    parse_dates=['RegisterDate'], 
                    infer_datetime_format=True
                   )
user_followers = pd.read_csv('./data/UserFollowers.csv', 
                             parse_dates=['CreationDate'], 
                             infer_datetime_format=True
                            )
# user_orgs = pd.read_csv('/kaggle/input/meta-kaggle/UserOrganizations.csv', 
#                         parse_dates=['JoinDate'], 
#                         infer_datetime_format=True
#                        )
user_ach = pd.read_csv('./data/UserAchievements.csv', 
                        parse_dates=['TierAchievementDate'], 
                        infer_datetime_format=True
                       )

# # Misc
# tags = pd.read_csv('/kaggle/input/meta-kaggle/Tags.csv')
# orgs = pd.read_csv('/kaggle/input/meta-kaggle/Organizations.csv', 
#                    parse_dates=['CreationDate'], infer_datetime_format=True
#                   )


#beep() # modified

In [ ]:
gms = (users[users['PerformanceTier'] == 4]
       .copy()
       .reset_index(drop=True)
       .drop(columns=['PerformanceTier'])
      )
#gms_ach = (user_ach[user_ach["Tier"]==4]
#        .copy()
#        .reset_index(drop=True)
#        .drop(columns=["UserId","AchievementType","Tier","TierAchievementDate"])
#        )
gms_ach = user_ach[user_ach["Tier"]==4][["UserId","AchievementType","Tier","TierAchievementDate"]]

In [ ]:
row = gms_ach.iloc[0]
print(gms_ach)

In [29]:
string = f"There are currently {gms.shape[0]} Grandmasters."
display(HTML(string))

In [30]:
def display_html(df, cols=None, num_rows=0):
    """Display columns cols and num_rows rows of the data 
    frame df in HTML.
    """
    
    if num_rows != 0:
        df_to_display = df.head(num_rows)
    else:
        df_to_display = df
    
    df_html = df_to_display.to_html(columns=cols, index=False, na_rep='',
                              escape=False, render_links=True)
    display(HTML(df_html))

In [ ]:
pklfl = './gmaster.pkl'

if os.path.exists(pklfl):
    with open(pklfl,'rb') as f:
        gms = pickle.load(f)

else:
    ii = 0
    for index in gms.index:
        ii += 1
        print(ii,'/',len(gms.index))
        time.sleep(2)
        row = gms.iloc[index]

        username = row.UserName
        profile_url = f'{KAGGLE_BASE_URL}{username}'
        displayname = row.DisplayName
        print(profile_url)

        result = requests.get(profile_url)
        src = result.text
        tmp = BeautifulSoup(src, 'html.parser').find_all("div", id="site-body")
        if tmp:
            soup = tmp[0].find("script")
        else:
            continue

        user_info = re.search('Kaggle.State.push\(({.*})', str(soup)).group(1)
        user_dict = json.loads(user_info)

        if 'country' in user_dict:
            country = user_dict['country']
            gms.loc[index, 'Country'] = country
        else:
            #country = ''
            if 'country' in locals():
                del country

        if 'city' in user_dict:
            city = user_dict['city']
            gms.loc[index, 'City'] = city
        else:
            #city = ''
            if 'city' in locals():
                del city
        
        if 'region' in user_dict:
            region = user_dict['region']
            gms.loc[index, 'Region'] = region
        else:
            #region = ''
            if 'region' in locals():
                del region
        
        if 'userAvatarUrl' in user_dict:
            avatar_url = user_dict['userAvatarUrl']
        else:
            #avatar_url = ''
            if 'avatar_url' in locals():
                del avatar_url
        
        if 'occupation' in user_dict:
            occupation = user_dict['occupation']
        else:
            #occupation = ''
            if 'occupation' in locals():
                del occupation
        
        if 'organization' in user_dict:
            organization = user_dict['organization']
            gms.loc[index, 'Occupation'] = organization
        else:
            #organization = ''
            if 'organization' in locals():
                del organization
        
        if 'followers' in user_dict:
            if 'count' in user_dict['followers']:
                num_followers = user_dict['followers']['count']
                gms.loc[index, 'NumFollowers'] = num_followers
            else:
                #num_followers = ''
                if 'num_followers' in locals():
                    del num_followers
        else:
            #num_followers = ''
            if 'num_followers' in locals():
                del num_followers
        
        if 'following' in user_dict:
            if 'count' in user_dict['following']:
                num_following = user_dict['following']['count']
                gms.loc[index, 'NumFollowing'] = num_following
            else:
                #num_following = ''
                if 'num_following' in locals():
                    del num_following
        else:
            #num_following = ''
            if 'num_following' in locals():
                del num_following
        
        if 'discussionsSummary' in user_dict:
            if 'totalResults' in user_dict['discussionsSummary']:
                num_posts = user_dict['discussionsSummary']['totalResults']
                gms.loc[index, 'NumPosts'] = num_posts
            else:
                #num_posts = ''
                if 'num_posts' in locals():
                    del num_posts
        else:
            #num_posts = ''
            if 'num_posts' in locals():
                del num_posts
        
        if 'datasetSummary' in user_dict:
            if 'totalResults' in user_dict['datasetsSummary']:
                num_datasets = user_dict['datasetsSummary']['totalResults']
                gms.loc[index, 'NumDatasets'] = num_datasets
            else:
                if 'num_datasets' in locals():
                    del num_datasets
        else:
            #num_datasets = ''
            if 'num_datasets' in locals():
                del num_datasets
        
        if 'scriptsSummary' in user_dict:
            if 'totalResults' in user_dict['scriptsSummary']:
                num_kernels = user_dict['scriptsSummary']['totalResults']
                gms.loc[index, 'NumKernels'] = num_kernels
            else:
                #num_kernels = ''
                if 'num_kernels' in locals():
                    del num_kernels
        else:
            if 'num_kernels' in locals():
                del num_kernels
        
        if 'totalResults' in user_dict['competitionsSummary']:
            num_comps = user_dict['competitionsSummary']['totalResults']
            gms.loc[index, 'NumCompetitions'] = num_comps
        else:
            #num_comps = ''
            if 'num_comps' in locals():
                del num_comps
        
        if 'linkedInUrl' in user_dict:
            linkedIn_url = user_dict['linkedInUrl']
        else:
            #linkedIn_url = ''
            if 'linkedIn_url' in locals():
                del linkedIn_url

        if 'twitterUserName' in user_dict:
            twitter_username = user_dict['twitterUserName']
        else:
            #twitter_username = ''
            if 'twitter_username' in locals():
                del twitter_username
            
        if 'gitHubUserName' in user_dict:
            github_username = user_dict['gitHubUserName']
        else:
            #github_username = ''
            if 'github_username' in locals():
                del github_username

        #print(gms_jpn)
        if profile_url and displayname and avatar_url:
            gms.loc[index, 'Image'] = f'<a href="{profile_url}" target="_blank" title="{displayname}"><img src="{avatar_url}" width="100" height="100"></a>'
        if avatar_url and avatar_url != '':
            gms.loc[index,'ImageUrl'] = f'= image("{avatar_url}")'
        if profile_url and profile_url != '':
            gms.loc[index,'ProfileUrl'] = f'= hyperlink("{profile_url}")'
        if 'twitter_username' in locals() and twitter_username and twitter_username != '':
            gms.loc[index, 'twitter'] = f'=hyperlink("{TWITTER_BASE_URL}{twitter_username}")'
            #print(gms.loc[index, 'twitter'])
        if 'github_username' in locals() and github_username and github_username != '':
            gms.loc[index, 'github'] = f'=hyperlink("{GITHUB_BASE_URL}{github_username}")'
        if 'linkedIn_url' in locals() and linkedIn_url and linkedIn_url != '':
            gms.loc[index, 'linkedin'] = f'=hyperlink("{linkedIn_url}")'
            
        #if occupation and organization:
        #    gms.loc[index, 'Occupation'] = f'{occupation} at {organization}'
        #elif organization:
        #    gms.loc[index, 'Occupation'] = organization
        #elif occupation:
        #    gms.loc[index, 'Occupation'] = occupation

    with open(pklfl,'wb') as f:
        pickle.dump(gms,f)

gms = gms.convert_dtypes()
gms_ach = gms_ach.convert_dtypes()
#display_html(gms, cols=['Id','DisplayName', 'RegisterDate', 'ImageUrl', 'ProfileUrl', 'twitter', 'github', 'linkedin', 'Country', 'Region', 'City', 'Occupation', 'NumFollowers'])

#gms.to_excel('./grandmasters.xlsx', sheet_name='GrandMasters',\
#             columns = ['Id','UserName','DisplayName','ImageUrl','ProfileUrl','RegisterDate','Country','City','Occupation','NumFollowers','linkedin','twitter','github'])

gms_ach.to_excel('./grandmasters_type.xlsx', sheet_name='GrandMasters',\
                columns = ['UserId','AchievementType','Tier','TierAchievementDate'])

beep() # modified

In [ ]:
#row = gms.iloc[0]
print(gms)

In [ ]:
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [51]:
geolocator = Photon(timeout=15)
geocode = RateLimiter(geolocator.geocode,
                      min_delay_seconds=3,
                      max_retries=3,
                      error_wait_seconds=30
                     )
m = folium.Map(location=[0, 30], tiles='OpenStreetMap', zoom_start=1)
mc = MarkerCluster()
locations = {}
for index in gms[gms.City.notna()].index:
    row = gms.iloc[index]
    displayname = row.DisplayName
    location = row.City
    popup = f'{displayname}; {location}'
    tooltip = f'{displayname}; {location}'
    
    if location in locations:
        lat, long = locations[location]
    else:
        point = geocode(location)
        lat, long = point.latitude, point.longitude
        locations[location] = [lat, long]
   
    mc.add_child(Marker([lat, long], popup=popup, tooltip=tooltip))
    
m.add_child(mc)

In [ ]:
# source: https://www.kaggle.com/code/danbraswell/kaggle-grandmaster-how-long-does-it-take


# Get libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import date
from IPython.display import Markdown

pd.set_option('display.max_columns', None)

now = datetime.now()
display(Markdown('This notebook is refreshed daily<br>Latest run: ' + now.strftime('%A %B %d, %Y') ))

# Read Meta-Kaggle files
if 'kaggle' in os.listdir("/"): # Running on kaggle
    df_users = pd.read_csv("./input/meta-kaggle/Users.csv")
    df_achieve = pd.read_csv("./input/meta-kaggle/UserAchievements.csv")
else: # Running locally    
    df_users = pd.read_csv("./data/Users.csv")
    #df_achieve = pd.read_csv("./data/UserAchievements.csv")
    df_achieve = user_ach

# Create new dataframe of only Grandmasters (tier 4) with a subset of columns
df_gm = df_achieve[df_achieve["Tier"]==4][["UserId","AchievementType","Tier","TierAchievementDate"]]

# The file UserAchievement.csv (read into df_achieve) contains the
# tier 4 achievements of GM's that are current Kaggle staff memebers.
# These are not included by Kaggle in their grandmaster count and hence
# will be removed from df_achieve.
# Note: "UserId" in df_achieve corresponds to "Id" in df_users.

# Make dataframe of kaggle staff members (denoted by tier=5 in Users.csv)
kaggle_staff = df_users[df_users.PerformanceTier==5]

# Remove the staff members from df_gm
for i in kaggle_staff.Id:
    df_gm = df_gm[df_gm.UserId != i]

# Function to get register date from df_users.
def get_register_date(UserId):
    register_date = df_users[df_users.Id==UserId][["RegisterDate"]].iloc[0,0]
    return register_date

# Create vector of register dates for every UserId in df_gm. Put in new column in df_gm.
vec=[]
for i in df_gm.UserId:
    vec.append( get_register_date(i) )
    
df_gm["RegisterDate"] = vec

# The date columns in df_gm are strings. Convert them to date objects.
df_gm["TierAchievementDate"] = pd.to_datetime(df_gm["TierAchievementDate"])
df_gm["RegisterDate"] = pd.to_datetime(df_gm["RegisterDate"])

# Compute the number of days between RegisterDate and TierAchievementDate and put in a new column in df_gm.
vec=[]
for date1,date2 in zip(df_gm.RegisterDate,df_gm.TierAchievementDate):
    delta = date2 - date1
    vec.append( delta.days )

df_gm["days_to_gm"] = vec

beep() # modified

In [ ]:
# Display stats and plot for first GM, any category.

# Some Users are GMs in more than 1 category. Here, we are interested in only the days to their first GM.
# So we'll sort by UserId and TierAchievementDate and remove a users entries after the first.
df_foo = df_gm.sort_values(by=["UserId","TierAchievementDate"])
df_gm_unique = df_foo.drop_duplicates(subset='UserId', keep="first")

# Define function to aid formatting output
def custom_string(string, weight="bold",size=16, lineheight=1.1, color=None):
    return f"<span style='font-size:{size}px; font-weight:{weight}; line-height:{lineheight}; color:{color}'>{string}</span>"
    
# Display stats
line0 = custom_string("Grandmaster, any category<br>",size=24,color="#ddaa17")
line1 = custom_string(f"Number Grandmasters = {len(df_gm_unique)}<br>")
line2 = custom_string(f"Min days to first GM    = {df_gm_unique['days_to_gm'].min()}<br>")
line3 = custom_string(f"Max days to first GM    = {df_gm_unique['days_to_gm'].max()}<br>")
line4 = custom_string(f"Mean days to first GM   = {df_gm_unique['days_to_gm'].mean():.0f}<br>")
line5 = custom_string(f"Median days to first GM = {df_gm_unique['days_to_gm'].median():.0f}")
display(Markdown(line0 + line1 + line2 + line3 + line4 + line5))
                      
# Make histogram of days needed for user to attain GM status, regardless of category.
ax = sns.histplot(df_gm_unique.days_to_gm,bins="auto",color="#ddaa17")
plt.title(f"Days to First Grandmaster (any category)")
plt.xlabel("Days")
plt.ylabel("Number of GMs")

# Put numbers on top of bars
for i in ax.containers:
    ax.bar_label(i,)

plt.show()

#plt.savefig('days_to_gm.png')

In [ ]:
# Display stats and plots for each GM category

# Define function to print stats and histogram on number of days taken to acquire GM status.
def process_achievement( category ):
    df_gm_new = df_gm[df_gm.AchievementType==category]

    category2 = category
    if category2 == "Scripts":
        category2 = "Notebooks"
        
    # Display stats
    line0 = custom_string(f"{category2} Grandmaster<br>",size=24,color="#ddaa17")
    line1 = custom_string(f"Number Grandmasters = {len(df_gm_new)}<br>")
    line2 = custom_string(f"Min days to GM    = {df_gm_new['days_to_gm'].min()}<br>")
    line3 = custom_string(f"Max days to GM    = {df_gm_new['days_to_gm'].max()}<br>")
    line4 = custom_string(f"Mean days to GM   = {df_gm_new['days_to_gm'].mean():.0f}<br>")
    line5 = custom_string(f"Median days to GM = {df_gm_new['days_to_gm'].median():.0f}")
    display(Markdown(line0 + line1 + line2 + line3 + line4 + line5))
    
    # Make histogram of days needed for user to attain GM status, regardless of category.
    ax = sns.histplot(df_gm_new.days_to_gm,bins="auto",color="#ddaa17")
    plt.title(f"Days to {category2} Grandmaster")
    plt.xlabel("Days")
    plt.ylabel("Number of GMs")

    # Put numbers on top of bars
    for i in ax.containers:
        ax.bar_label(i,)

    plt.show()

# Display results for each category    
categories = ["Competitions","Datasets","Scripts","Discussion"]    
for cat in categories:
    process_achievement( cat ) 

In [42]:
print(gms)

           Id          UserName               DisplayName RegisterDate
0         808   alexxanderlarko          Alexander  Larko   2010-05-14
1        1335          salimali                 Sali Mali   2010-06-22
2        1455           mjahrer            Michael Jahrer   2010-06-25
3        1483    yassertabandeh           Yasser Tabandeh   2010-06-27
4        1602           chefele        Christopher Hefele   2010-07-01
..        ...               ...                       ...          ...
422  10862884           arezalo             AmirrezaLotfi   2022-06-20
423  11112582   nguyenthicamlai        Nguyen Thi Cam Lai   2022-07-22
424  11347907   soumendraprasad  SOUMENDRA PRASAD MOHANTY   2022-08-20
425  11570669  faisaljanjua0555               Mian Faisal   2022-09-12
426  12064410        justin2028                 Justin Oh   2022-10-23

[427 rows x 4 columns]


In [47]:
if 'gms' in locals():
    print('gmsは存在します')
else:
    print('gmsは存在しません')

gmsは存在しません
